In [ ]:
import sys
sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import yaml
import core
import keras
import core.regression_models.RegressionTransformer as RegressionTransformer


PLOTS_DIR = f"plots/regresion_transformer/"
MODEL_DIR = f"models/regresion_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

data_config = DataProcessor.load_from_npz(
    data_configs["data_path"]["nominal"], max_events=4_000_000
)

X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)

2025-12-18 18:23:42.243155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766078622.265511 1414489 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766078622.272829 1414489 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766078622.290765 1414489 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766078622.290782 1414489 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766078622.290785 1414489 computation_placer.cc:177] computation placer alr

In [2]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatTransformer(data_config)

In [3]:
Transformer.build_model(
    hidden_dim=128,
    num_heads=8,
    central_transformer_stack_size=4,
    regression_transformer_stack_size=4,
    dropout_rate=0.1,
    compute_HLF=True,
    log_variables=True,
)

Transformer.adapt_normalization_layers(X_train)

Transformer.compile_model(
    optimizer=keras.optimizers.Lion(learning_rate=1e-4),
    loss={
        "normalized_regression": keras.losses.MeanSquaredLogarithmicError(),
        "assignment": core.utils.AssignmentLoss(),
    },
    metrics={
        "normalized_regression": core.utils.RegressionDeviation(),
        "assignment": core.utils.AssignmentAccuracy(),
    },
    loss_weights={
        "normalized_regression": 1.0,
        "assignment": 5.0,
    },
)


I0000 00:00:1766078802.649026 1414489 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Submodel inputs:  [<KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs>, <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs>, <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs>]
Input tensor:  <KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs> jet_inputs
Input tensor:  <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs> lep_inputs
Input tensor:  <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs> met_inputs


I0000 00:00:1766078805.778394 1417754 service.cc:152] XLA service 0x7f3ee0002db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1766078805.778422 1417754 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-12-18 18:26:45.803609: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1766078805.858866 1417754 cuda_dnn.cc:529] Loaded cuDNN version 91500


 143/1697 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

I0000 00:00:1766078806.151020 1417754 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1697/1697 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Adapted normalization layer:  hlf_input_normalization
Submodel inputs:  [<KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs>, <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs>, <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs>]
Input tensor:  <KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs> jet_inputs
Input tensor:  <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs> lep_inputs
Input tensor:  <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs> met_inputs
  50/1697 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step    

2025-12-18 18:26:50.523451: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


1697/1697 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


2025-12-18 18:26:52.358611: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


Adapted normalization layer:  met_input_normalization
Submodel inputs:  [<KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs>, <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs>, <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs>]
Input tensor:  <KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs> jet_inputs
Input tensor:  <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs> lep_inputs
Input tensor:  <KerasTensor shape=(None, 1, 2), dtype=float32, sparse=False, ragged=False, name=met_inputs> met_inputs
1697/1697 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Adapted normalization layer:  lep_input_normalization
Submodel inputs:  [<KerasTensor shape=(None, 6, 5), dtype=float32, sparse=False, ragged=False, name=jet_inputs>, <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, na

In [4]:
Transformer.train_model(
    X_train,
    y_train,
    batch_size=1024,
    epochs=5,
    copy_data=True,
)

Epoch 1/5


2025-12-18 18:27:20.960921: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-18 18:27:51.563176: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_8', 44 bytes spill stores, 44 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_5', 44 bytes spill stores, 44 bytes spill loads



2807/2809 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - assignment_accuracy: 0.6246 - assignment_loss: 0.0918 - loss: 0.5640 - normalized_regression_deviation: 0.5483 - normalized_regression_loss: 0.1052

2025-12-18 18:30:10.968540: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2809/2809 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - assignment_accuracy: 0.6247 - assignment_loss: 0.0918 - loss: 0.5639 - normalized_regression_deviation: 0.5483 - normalized_regression_loss: 0.1052

2025-12-18 18:30:43.822950: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-18 18:30:52.638964: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2809/2809 ━━━━━━━━━━━━━━━━━━━━ 232s 65ms/step - assignment_accuracy: 0.6247 - assignment_loss: 0.0917 - loss: 0.5639 - normalized_regression_deviation: 0.5483 - normalized_regression_loss: 0.1052 - val_assignment_accuracy: 0.7902 - val_assignment_loss: 0.0606 - val_loss: 0.3687 - val_normalized_regression_deviation: 0.4978 - val_normalized_regression_loss: 0.0658
Epoch 2/5
2809/2809 ━━━━━━━━━━━━━━━━━━━━ 143s 51ms/step - assignment_accuracy: 0.7776 - assignment_loss: 0.0628 - loss: 0.3875 - normalized_regression_deviation: 0.5094 - normalized_regression_loss: 0.0734 - val_assignment_accuracy: 0.7982 - val_assignment_loss: 0.0592 - val_loss: 0.3563 - val_normalized_regression_deviation: 0.4886 - val_normalized_regression_loss: 0.0604
Epoch 3/5
2809/2809 ━━━━━━━━━━━━━━━━━━━━ 144s 51ms/step - assignment_accuracy: 0.7886 - assignment_loss: 0.0610 - loss: 0.3728 - normalized_regression_deviation: 0.5012 - normalized_regression_loss: 0.0680 - val_assignment_accuracy: 0.8037 - val_assignment_l

In [5]:
Transformer.save_model(MODEL_DIR + "model.keras")

Model saved to models/regresion_transformer/model.keras


In [6]:
neutrinos = Transformer.reconstruct_neutrinos(X_val)

2025-12-18 18:40:34.271872: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-18 18:40:51.924005: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1/met_input_transform_1/assert_equal_1/Assert/Assert


In [12]:
Transformer.evaluate_regression(X_val, y_val["neutrino_truth"])

KeyboardInterrupt: 